In [ ]:
import shutil

@patch("rss_collector.orchestration.get_proxy_settings")
@patch("rss_collector.orchestration.fetch_feed_content_with_proxy")
@patch("rss_collector.orchestration.parse_feed")
def test_process_feeds(mock_parse_feed, mock_fetch_feed, mock_get_proxy_settings, mock_feed_data, mock_proxy_settings, test_feed_urls):
    """
    Test the process_feeds function.
    """
    # Mock dependencies
    mock_get_proxy_settings.return_value = mock_proxy_settings
    mock_fetch_feed.return_value = "Mock RSS Feed Content"
    mock_parse_feed.return_value = mock_feed_data

    # Ensure the rss_metadata directory is empty
    output_dir = "rss_metadata"
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # Call process_feeds
    from rss_collector.orchestration import process_feeds
    process_feeds(test_feed_urls)

    # Assertions
    saved_files = os.listdir(output_dir)
    assert len(saved_files) == 1, f"Expected one metadata file, but found {len(saved_files)}."
    metadata_file = os.path.join(output_dir, saved_files[0])

    # Validate the contents of the metadata file
    with open(metadata_file, "r") as f:
        metadata = json.load(f)

    assert "timestamp" in metadata, "Metadata is missing the timestamp key."
    assert "articles" in metadata, "Metadata is missing the articles key."
    assert len(metadata["articles"]) == len(mock_feed_data["entries"]), (
        f"Expected {len(mock_feed_data['entries'])} articles, but found {len(metadata['articles'])}."
    )

    # Cleanup after the test
    shutil.rmtree(output_dir)
